# Load Balancer Performance Analysis
This notebook provides insights on the performance of the implemented load balancer.

## Required Libraries

In [6]:
# Asynchronous requests handling
import asyncio
import aiohttp

# Frequency counter
from collections import Counter

# For random operations - such as random request IDs
import random

# Visualization
import matplotlib.pyplot as plt

# For creating the consistent hash ring
from consistent_hash import ConsistentHashRing

## Constants

In [7]:
NUM_REQUESTS = 10000 # Number of async requests to be executed

## A-1: Requests Count Per Server Instance
- This experiment involves executing 10,000 async requests on the 3 server containers.
- Requests count are displayed in a bar chart.

### Asynchronous launching of 10,000 requests
- The launching of 10,000 is achieved by using the AIOHTTP Python library, which enables for asynchronous HTTP client/server communications.
- It is built on top of asyncio, which handles non-blocking I/O operations, allowing for concurrency.

In [8]:
URL = "http://localhost:5000/home"

async def fetch(session, url=URL):
    """Gets a response from the server URL, returning the server id."""
    try:
        async with session.get(url) as response:
            data = await response.json()
            message = data.get("message", "")
            server_id = message.split()[-1].split(":")[0]
            return server_id
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None


async def count_requests_per_server():
    """Counts the number of requests handled by each server."""
    counter = Counter()
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url=URL) for _ in range(NUM_REQUESTS)]
        results = await asyncio.gather(*tasks)
        for server in results:
            if server is not None:
                counter[server] += 1

    return counter

### Requests Distribution Bar Chart

In [9]:
async def main():
    """Plots a bar chart showing the number of requests per server."""
    requests_count = await count_requests_per_server()
    servers = sorted(requests_count.keys())
    counts = list(requests_count.values())

    plt.bar(servers, counts)
    plt.title("Request distribution Across Servers")
    plt.xlabel("Servers")
    plt.ylabel("Requests")
    plt.show()

await main()

Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:5000/home: 
Error fetching http://localhost:

AttributeError: module 'numpy' has no attribute 'logical_or'

<Figure size 640x480 with 0 Axes>

### Observations
From the graph, most of the requests (more than 90%) are handled by server1, with the rest handling a fairly even number of requests. This may be caused by the hash function mapping requests to servers, which may be returning hash values that direct requests to server1. This represents an imbalance in the number of requests handled across the 3 servers, as server1 handles a large number of requests, leaving the rest of the servers under-utilised.

## A-2: Average Load of Servers for N = 2 to 6

- This experiment involves iteratively increasing the number of servers from 2 to 6 while launching async requests.

In [10]:
SERVER_RANGE = range(2, 7)
AVERAGES = []

async def fetch_server(ring):
    """Gets the server for a particular request."""
    request_id = random.randint(100000, 999999)
    server = ring.get_server_for_request(request_id)
    return server


async def run_simulation(num_servers):
    """Creates a hash ring and simulates async requests for a set number of servers."""
    ring = ConsistentHashRing(num_servers=num_servers, slots=512, virtual_nodes=9)
    counter = Counter()

    # Count requests per server
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_server(ring) for _ in range(NUM_REQUESTS)]
        results = await asyncio.gather(*tasks)
        for server in results:
            counter[server] += 1

    loads = list(counter.values())
    avg_load = sum(loads) / num_servers
    return avg_load

## A Line Chart Showing Average Load against No. of Servers

In [11]:
async def main():
    """Plots the average load against number of servers N."""
    for N in SERVER_RANGE:
        avg = await run_simulation(N)
        print(f"N={N}, Avg load: {avg:.2f}")
        AVERAGES.append(avg)

    plt.plot(list(SERVER_RANGE), AVERAGES, marker='o')
    plt.title("Avg Load against Number of Servers")
    plt.xlabel("Servers (N)")
    plt.ylabel("Avg Load")
    plt.grid()
    plt.show()

await main()

N=2, Avg load: 5000.00
N=3, Avg load: 3333.33
N=4, Avg load: 2500.00
N=5, Avg load: 2000.00
N=6, Avg load: 1666.67


AttributeError: module 'numpy' has no attribute 'logical_or'

<Figure size 640x480 with 0 Axes>